<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/24task3_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requir

In [ ]:
!pip install -U sentence-transformers rank_bm25

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
from datasets import load_dataset
ds = load_dataset("florentgbelidji/car-reviews")

Generating train split:   0%|          | 0/36984 [00:00<?, ? examples/s]

In [ ]:
data = ds['train']
df = pd.DataFrame(data)

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5
1,1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5
2,2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5
3,3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",5
4,4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5


In [ ]:
df.shape

(36984, 8)

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

(36984, 8)

In [ ]:
df_copy=df.copy()
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5
1,1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5
2,2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5
3,3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",5
4,4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5


In [ ]:
import re
import string

df_copy['reviews'] =  "Title: " + df_copy.Review_Title.str.strip() +"; Review Text: " + df_copy.Review.str.strip()
df_copy['reviews'] = df_copy['reviews'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
df_copy['reviews'] = df_copy['reviews'].str.lower()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,reviews
0,0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5,title great vehicle toyota best design ever th...
1,1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5,title my 4th previa best van ever made review ...
2,2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5,title moms taxi babies ride review text sold 8...
3,3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",5,title my favorite van ever review text i have ...
4,4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5,title best minivan ever review text my 1997 aw...


In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
sample =df_copy.sample(50)

In [ ]:
sample['embedding'] = sample['reviews'].apply(lambda x: model.encode(x))


In [ ]:
!mkdir /content/drive/MyDrive/Task3

mkdir: cannot create directory ‘/content/drive/MyDrive/Task3’: File exists


In [ ]:
sample.to_pickle('/content/drive/MyDrive/Task3/cars_sample.pkl')

In [ ]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Task3/cars_sample.pkl')

In [ ]:
!pip install openai==0.27.7
!pip install gradio
!pip install -U sentence-transformers

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
def search_reviews(df, query, n=10, pprint=True):

    query_embedding = model.encode(query,show_progress_bar=True)

    sample["similarity"] = sample.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1))) #similarity against each doc

    results = (
        sample.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [ ]:
query = 'comfortable and reliable'


In [ ]:
results = search_reviews(sample, query, n=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,Unnamed: 0.1,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,reviews,embedding,similarity
7859,10544,8301,on 08/01/10 21:07 PM (PDT),Orin Beitlich,2005 Toyota RAV4 SUV Fwd 4dr SUV (2.4L 4cyl 4A),Best Car,This was my first Toyota ever and I wished I ...,5,title best car review text this was my first t...,"[-0.056641582, -0.02991242, 0.039934646, 0.068...",[0.4205275]
23844,7951,6243,on 02/23/10 11:27 AM (PST),steve,2009 BMW 3 Series Coupe 335i xDrive 2dr Coupe ...,Best Car I have Owned,This car is great. First BMW I have owned an...,5,title best car i have owned review text this c...,"[-0.056689743, -0.016712746, 0.034581564, 0.06...",[0.40390813]
7946,10631,8388,on 09/29/05 12:57 PM (PDT),RoninNC66,2004 Toyota RAV4 SUV AWD 4dr SUV (2.4L 4cyl 5M),Fun Reliability,Waited over a year to FINALLY locate my RED A...,5,title fun reliability review text waited over ...,"[-0.026882017, -0.031043813, 0.0137463175, 0.0...",[0.39496544]
20627,3486,2805,on 02/13/06 16:12 PM (PST),dharods,2006 BMW X3 SUV 3.0i 4dr SUV AWD (3.0L 6cyl 6M),Superb,It's surely the Ultimate Driving Machine. It ...,5,title superb review text its surely the ultima...,"[-0.032754242, -0.022711465, 0.025687426, 0.06...",[0.3939399]
6128,8554,6525,on 07/29/15 19:28 PM (PDT),Bob Sandon,2009 Toyota Prius Hatchback Standard 4dr Hatch...,"Magnificent, but Dreadfully Boring","I have put approximately 143,000 miles on thi...",4,title magnificent but dreadfully boring review...,"[-0.05790357, 0.026864443, 0.046435956, 0.0736...",[0.3926799]


In [ ]:
def search(query):
  n = 10

  query_embedding = model.encode(query)

  sample["similarity"] = sample.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      sample.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.Vehicle_Title[r] not in hlist:
          smalldf = results.loc[results.Vehicle_Title == results.Vehicle_Title[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "Vehicle_Title":results.Vehicle_Title[r],
            "Review_Title":results.Review_Title[r],
            "Review":results.Review[r],

          })
          hlist.append(results.Vehicle_Title[r])
  return resultlist




In [ ]:
from google.colab import userdata
import openai
openai.api_key = userdata.get('OPENAI_API_KEY1')

In [ ]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
  focusing on which is the top Vehicles based on the query. You should responsd as you are an expert in Vehicles. Make sure to address the user as Sara.
  Make the output clean and concise.

    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.2, #higher temperature means more creative or more hallucination
        messages = messages


    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message['content'].strip()

    return generated_text

In [ ]:

import gradio as gr

# def search(query):
#   n = 15
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

#   results = (
#       df.sort_values("similarity", ascending=False)
#       .head(n))

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           smallarr = smalldf.similarity[r].max()
#           sm =smalldf.rating[r].mean()

#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "description":results.description[r],
#             "relevance score": smallarr.tolist(),
#             "rating": sm.tolist(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist

def greet(query):
    bm25 = generate_answer(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)
# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://22a3213f96703ca384.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
